In [ ]:
# -- Imports
import os
import cv2
import pandas as pd
from dermaml import features
from dermaml import image

# from dermaml import data
# import PIL
# from PIL import Image
# import mlflow # creates error
import matplotlib.pyplot as plt
# import skimage

# -- Feature Eng Imports
import numpy as np
import skimage
import pyfeats


In [ ]:
# -- Read the CSV file
df = pd.read_csv(root+ csv_file)
dorsals = df.loc[df['aspectOfHand'] == 'dorsal right']
dorsals = dorsals.drop_duplicates(subset=['id'])

## -- subset images further: 3 of each skin color
dorsals = dorsals.groupby('skinColor').tail(3)

In [ ]:
# -- Read Images

def read_local(image_fnames, image_dir):
    images = []
    for filename in image_fnames:
        img = cv2.imread(os.path.join(image_dir, filename))
        if (img is not None) & (len(img.shape) > 2):
            images.append(img)
    return images

# ds = hub.load('hub://activeloop/11k-hands')
# images = ds.images

image_fnames = dorsals.loc[:, 'imageName']
images = read_local(image_fnames, root+image_folder)

In [ ]:
# -- Image Preprocessing (brightness)

def remove_brightness(image):
    '''
    Converts an RGB-channeled image to HSV/HSB and removes the 'value' or 'brightness' channel.

    Arguments
    ---------
    `image`: an RGB numpy array
    '''
    assert len(image.shape) == 3
    hsv_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    hsv_image[:, :, 2] = 0
    return hsv_image

iso_images = [image.remove_background(im) for im in images]
hs_images = [remove_brightness(im) for im in images]


In [ ]:
im_ind = 2
bw_im_gray = cv2.cvtColor(images[im_ind], cv2.COLOR_RGB2GRAY)
bw_im = hs_images[im_ind][:,:,1]
test_im = bw_im_gray.copy()

kernel = np.ones((5,5),np.float32)/25
blurred_image = cv2.filter2D(bw_im,-1,kernel)

th3 = cv2.adaptiveThreshold(blurred_image,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
        cv2.THRESH_BINARY,11,2)

contours, hierarchy = cv2.findContours(th3,  
    cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) 

largest_area =  np.argmax([cv2.contourArea(cnt) for cnt in contours])
cnt = contours[largest_area]

draw = cv2.drawContours(test_im, contours, -1, 1, thickness=cv2.FILLED)

# plt.imshow(th3*255)
# plt.imshow(draw)

M = cv2.moments(cnt)
if M['m00'] != 0:
    cx = int(M['m10']/M['m00'])
    cy = int(M['m01']/M['m00'])
# draw_bw = cv2.cvtColor(draw, cv2.COLOR_RGB2GRAY)
seed = (cx,cy)
h,w = draw.shape
mask = np.zeros((h+2,w+2),np.uint8)

floodflags = 4
floodflags |= cv2.FLOODFILL_MASK_ONLY
floodflags |= (255 << 8)

num,im,mask,rect = cv2.floodFill(draw, mask, seed, (255,0,0), (10,)*3, (10,)*3, floodflags)

fig, ax = plt.subplots(1,3)
ax[0].imshow(bw_im)
ax[1].imshow(draw)
ax[2].imshow(mask)